In [1]:
# 🛠️ Step 1: Set up Spark Session
# Install pyspark if not already installed
%pip install pyspark

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("MallCustomersAnalysis").getOrCreate()

print("✅ Spark Session Created")
print(f"Spark Version: {spark.version}")

Note: you may need to restart the kernel to use updated packages.


your 131072x1 screen size is bogus. expect trouble
25/04/29 23:18:46 WARN Utils: Your hostname, HKCP resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/04/29 23:18:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/29 23:18:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


✅ Spark Session Created
Spark Version: 3.5.5


In [ ]:
# 🛠️ Step 2: Load the Dataset
# Load the dataset into a DataFrame
df = spark.read.csv("/mnt/c/Users/hmdkr/DE-Projects/mall-customers/data/Mall_Customers.csv", header=True, inferSchema=True)
print("✅ Dataset Loaded")
print(f"Number of Rows: {df.count()}")
print(f"Number of Columns: {len(df.columns)}")
print(f"Columns: {df.columns}")
print("Sample Data:")
df.show(5) 





✅ Dataset Loaded
Number of Rows: 200
Number of Columns: 5
Columns: ['CustomerID', 'Genre', 'Age', 'Annual Income (k$)', 'Spending Score (1-100)']
Sample Data:
+----------+------+---+------------------+----------------------+
|CustomerID| Genre|Age|Annual Income (k$)|Spending Score (1-100)|
+----------+------+---+------------------+----------------------+
|         1|  Male| 19|                15|                    39|
|         2|  Male| 21|                15|                    81|
|         3|Female| 20|                16|                     6|
|         4|Female| 23|                16|                    77|
|         5|Female| 31|                17|                    40|
+----------+------+---+------------------+----------------------+
only showing top 5 rows



In [17]:
# 🛠️ Step 3: Data Preprocessing
print("✅ Summary Statistics:")
df.describe().show()
print("✅ Data Types:") 
# Print the schema of the DataFrame to see data types
# This will show the data types of each column in the DataFrame
df.printSchema()
# Check for missing values
from pyspark.sql.functions import col, sum
# Check for missing values in each column
# This will create a new DataFrame with the count of missing values for each column
missing_values = df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])
print("✅ Missing Values Checked")
missing_values.show()

✅ Summary Statistics:
+-------+------------------+------+-----------------+------------------+----------------------+
|summary|        CustomerID| Genre|              Age|Annual Income (k$)|Spending Score (1-100)|
+-------+------------------+------+-----------------+------------------+----------------------+
|  count|               200|   200|              200|               200|                   200|
|   mean|             100.5|  NULL|            38.85|             60.56|                  50.2|
| stddev|57.879184513951124|  NULL|13.96900733155888| 26.26472116527124|    25.823521668370173|
|    min|                 1|Female|               18|                15|                     1|
|    max|               200|  Male|               70|               137|                    99|
+-------+------------------+------+-----------------+------------------+----------------------+

✅ Data Types:
root
 |-- CustomerID: integer (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Age: inte

In [ ]:
# 📊 Step 4: Analysis 

# 4.1 Spending Score Distribution
df.groupBy("Spending Score (1-100)") \
  .count() \
  .orderBy("Spending Score (1-100)") \
  .show(20)

+----------------------+-----+
|Spending Score (1-100)|count|
+----------------------+-----+
|                     1|    2|
|                     3|    1|
|                     4|    2|
|                     5|    4|
|                     6|    2|
|                     7|    1|
|                     8|    1|
|                     9|    1|
|                    10|    2|
|                    11|    1|
|                    12|    1|
|                    13|    3|
|                    14|    4|
|                    15|    3|
|                    16|    2|
|                    17|    3|
|                    18|    1|
|                    20|    2|
|                    22|    1|
|                    23|    1|
+----------------------+-----+
only showing top 20 rows



In [19]:
# 4.2 Average Spending Score by Gender 
df.groupBy("Genre") \
  .agg({"Spending Score (1-100)": "avg"}) \
  .show()


+------+---------------------------+
| Genre|avg(Spending Score (1-100))|
+------+---------------------------+
|Female|         51.526785714285715|
|  Male|          48.51136363636363|
+------+---------------------------+



In [20]:
# 📈 4.3: Average Income by Spending Score
from pyspark.sql.functions import avg, count, when
df = df.withColumn(
    "Age_Group",
    when(df.Age < 20, "Teenagers")
    .when((df.Age >= 20) & (df.Age < 30), "20s")
    .when((df.Age >= 30) & (df.Age < 40), "30s")
    .when((df.Age >= 40) & (df.Age < 50), "40s")
    .otherwise("50+")  
)
df.groupBy("Age_Group") \
  .agg(avg("Spending Score (1-100)").alias("Avg Spending Score"), count("*").alias("Count")) \
  .orderBy("Age_Group") \
  .show()

+---------+------------------+-----+
|Age_Group|Avg Spending Score|Count|
+---------+------------------+-----+
|      20s| 61.95348837209303|   43|
|      30s| 61.09836065573771|   61|
|      40s| 34.94871794871795|   39|
|      50+|              38.4|   45|
|Teenagers|              46.5|   12|
+---------+------------------+-----+



In [21]:
# 4.4 High-Spending Customers 
df.orderBy(df["Spending Score (1-100)"].desc()).show(5)
# 4.5 Low-Spending Customers
df.orderBy(df["Spending Score (1-100)"].asc()).show(5)
# 4.6 High-Income Customers
df.orderBy(df["Annual Income (k$)"].desc()).show(5)
# 4.7 Low-Income Customers
df.orderBy(df["Annual Income (k$)"].asc()).show(5)


+----------+------+---+------------------+----------------------+---------+
|CustomerID| Genre|Age|Annual Income (k$)|Spending Score (1-100)|Age_Group|
+----------+------+---+------------------+----------------------+---------+
|        12|Female| 35|                19|                    99|      30s|
|        20|Female| 35|                23|                    98|      30s|
|       146|  Male| 28|                77|                    97|      20s|
|       186|  Male| 30|                99|                    97|      30s|
|       128|  Male| 40|                71|                    95|      40s|
+----------+------+---+------------------+----------------------+---------+
only showing top 5 rows

+----------+-----+---+------------------+----------------------+---------+
|CustomerID|Genre|Age|Annual Income (k$)|Spending Score (1-100)|Age_Group|
+----------+-----+---+------------------+----------------------+---------+
|       157| Male| 37|                78|                     1|  

In [ ]:
# 4.8 Spending Score Categories 
df = df.withColumn(
    "Spending_Category",
    when(df["Spending Score (1-100)"] < 20, "Low")
    .when((df["Spending Score (1-100)"] >= 20) & (df["Spending Score (1-100)"] < 50), "Medium")
    .when((df["Spending Score (1-100)"] >= 50) & (df["Spending Score (1-100)"] < 80), "High")
    .otherwise("Very High")  
)
df.groupBy("Spending_Category") \
  .agg(avg("Annual Income (k$)").alias("Avg Annual Income"), count("*").alias("Count")) \
  .orderBy("Spending_Category") \
  .show()

+-----------------+-----------------+-----+
|Spending_Category|Avg Annual Income|Count|
+-----------------+-----------------+-----+
|             High|             56.5|   72|
|              Low|65.17647058823529|   34|
|           Medium|          58.6875|   64|
|        Very High|69.06666666666666|   30|
+-----------------+-----------------+-----+



In [ ]:
# 4.9: Average Income by Spending Score
df.groupBy("Spending Score (1-100)") \
  .agg(avg("Annual Income (k$)").alias("Avg Income"), count("*").alias("Count")) \
  .orderBy("Spending Score (1-100)") \
  .show(20)

+----------------------+-----------------+-----+
|Spending Score (1-100)|       Avg Income|Count|
+----------------------+-----------------+-----+
|                     1|             78.0|    2|
|                     3|             19.0|    1|
|                     4|             31.5|    2|
|                     5|             63.5|    4|
|                     6|             17.0|    2|
|                     7|             73.0|    1|
|                     8|            113.0|    1|
|                     9|             71.0|    1|
|                    10|             80.5|    2|
|                    11|             71.0|    1|
|                    12|             77.0|    1|
|                    13|             65.0|    3|
|                    14|            43.25|    4|
|                    15|68.66666666666667|    3|
|                    16|             99.0|    2|
|                    17|71.66666666666667|    3|
|                    18|            137.0|    1|
|                   

In [24]:
# 4.10 Gender Distribution in Spending Categories
df.groupBy("Spending_Category", "Genre") \
  .agg(count("*").alias("Count")) \
  .orderBy("Spending_Category", "Genre") \
  .show()

+-----------------+------+-----+
|Spending_Category| Genre|Count|
+-----------------+------+-----+
|             High|Female|   42|
|             High|  Male|   30|
|              Low|Female|   13|
|              Low|  Male|   21|
|           Medium|Female|   41|
|           Medium|  Male|   23|
|        Very High|Female|   16|
|        Very High|  Male|   14|
+-----------------+------+-----+



In [25]:
# 4.11 Annual Income by Spending Category
df.groupBy("Spending_Category") \
  .agg(avg("Annual Income (k$)").alias("Avg Annual Income"), count("*").alias("Count")) \
  .orderBy("Spending_Category") \
  .show()

+-----------------+-----------------+-----+
|Spending_Category|Avg Annual Income|Count|
+-----------------+-----------------+-----+
|             High|             56.5|   72|
|              Low|65.17647058823529|   34|
|           Medium|          58.6875|   64|
|        Very High|69.06666666666666|   30|
+-----------------+-----------------+-----+



In [26]:
# 4.12 Age Group by Spending Category
df.groupBy("Spending_Category", "Age_Group") \
  .agg(count("*").alias("Count")) \
  .orderBy("Spending_Category", "Age_Group") \
  .show()

+-----------------+---------+-----+
|Spending_Category|Age_Group|Count|
+-----------------+---------+-----+
|             High|      20s|   21|
|             High|      30s|   24|
|             High|      40s|    6|
|             High|      50+|   16|
|             High|Teenagers|    5|
|              Low|      20s|    3|
|              Low|      30s|    7|
|              Low|      40s|   11|
|              Low|      50+|   11|
|              Low|Teenagers|    2|
|           Medium|      20s|    8|
|           Medium|      30s|   13|
|           Medium|      40s|   21|
|           Medium|      50+|   18|
|           Medium|Teenagers|    4|
|        Very High|      20s|   11|
|        Very High|      30s|   17|
|        Very High|      40s|    1|
|        Very High|Teenagers|    1|
+-----------------+---------+-----+



In [27]:
# 4.13 Common Profiles of High Spenders
df.filter(df["Spending Score (1-100)"] > 80) \
  .groupBy("Genre", "Age_Group") \
  .agg(count("*").alias("Count")) \
  .orderBy("Count", ascending=False) \
  .show(10)

+------+---------+-----+
| Genre|Age_Group|Count|
+------+---------+-----+
|Female|      30s|    9|
|  Male|      30s|    8|
|Female|      20s|    7|
|  Male|      20s|    4|
|  Male|Teenagers|    1|
|  Male|      40s|    1|
+------+---------+-----+



In [28]:
# 4.14 Correlation Analysis
df.stat.corr("Annual Income (k$)", "Spending Score (1-100)")
df.stat.corr("Age", "Spending Score (1-100)")
df.stat.corr("Age", "Annual Income (k$)")

-0.01239804273606026

## ✅ Final Conclusion

This analysis of the Mall Customers dataset revealed distinct spending behavior patterns across different income groups and ages. While income alone does not fully explain spending behavior, certain segments — particularly those with high spending scores and moderate-to-high income — show strong potential for targeted marketing. 

The use of PySpark enabled scalable and efficient processing, and the modular project structure supports future extensions, including clustering and predictive modeling. This project demonstrates foundational data engineering and analysis skills suitable for real-world customer segmentation problems.
